In [ ]:
#!pip install python-terrier

import numpy as np
import pandas as pd
import pyterrier as pt
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
if not pt.started():
    pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])



PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/M1_S2/RITAL/projet2
import json
import os
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/M1_S2/RITAL/projet2


In [ ]:
!pip install python-terrier
import pandas as pd

import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

# Chargement des fichiers

### QA

On génére les fichiers :
- <code>relevant</code> : dataframe des passages pertinents pour chaque requete
- <code>train_queries</code>, <code>dev_queries</code>, <code>test_queries</code> : les dataframe de reqête

Les docno sont sous la forme {numéro doc},{numéro passage}

In [ ]:
def string_to_list (s):
    """
    Converti un string de liste en une liste d'entier
    ex "21,22"->[21,22]
    """
    l=[]
    for i in s.split(',') :
        l.append(int(i))
    return l

In [ ]:
relevant = pd.DataFrame({'qid' : pd.Series(dtype='int'), 'docno' : pd.Series(dtype='str'), 'iteration' : pd.Series(dtype='float')})

def load_tsv (fname) :
    global relevant
    
    queries = pd.DataFrame({'qid' : pd.Series(dtype='int'), 'query' : pd.Series(dtype='str')})
    
    data = pd.read_csv(fname+'.tsv', sep='\t')
    for i in range(len(data)) :
        #ajout dans queries
        tupl=pd.DataFrame({'qid' : [data['QID'][i]],
                                      'query' : [data['Question'][i]]})
        queries=pd.concat([queries,tupl])
        
        #ajout dans relevant
        passages = string_to_list(data['RelevantPassages'][i])
        tupl=pd.DataFrame({'qid' : [data['QID'][i] for i in passages],
                           'docno' : [f"{data['DocumentID'][i]},{p}" for p in passages],
                           'iteration' : [1 for i in passages]})
        relevant=pd.concat([relevant,tupl])
    return queries



train_queries=load_tsv("data/train").reset_index().drop(columns=['index'])
dev_queries=load_tsv("data/dev").reset_index().drop(columns=['index'])
test_queries=load_tsv("data/test").reset_index().drop(columns=['index'])

relevant=relevant.reset_index().drop(columns=['index'])

In [ ]:
relevant

,qid,docno,iteration
0,956,"672,4",1.0
1,195,"359,1",1.0
2,557,"359,2",1.0
3,956,"285,4",1.0
4,2610,"579,14",1.0
...,...,...,...
6962,782,"393,2",1.0
6963,3738,"829,4",1.0
6964,844,"829,5",1.0
6965,1140,"364,1",1.0


In [ ]:
train_queries

,qid,query
0,3086,What is the role of conversionism in Evangelic...
1,195,How did the assault on the Bastille the first ...
2,557,What is the prehistory of Albania?
3,1508,What significance did Bulgaria have in the end...
4,956,What is the rationale of support of the Common...
...,...,...
3327,3700,How Do We Separate The Gray Wolf Or Grey Wolf ...
3328,1588,What is Greenland's self governing act?
3329,1971,How did conquering of Alexander the great effe...
3330,1153,What is Amnesty International?


### Documents

On charge le json dans un fichier pyterrier index

In [3]:
import json
pt_index_path = './passageQA'
if not os.path.exists(pt_index_path + "/data.properties") :
    # création de l'index
    indexer = pt.DFIndexer(pt_index_path, overwrite=True) 

    # chargement du json en dataframe
    with open('WikiPassageQA/document_passages.json', 'r') as f:
        data_row = json.load(f)
    data={"docno" : [k0+','+k1 for k0,v0 in data_row.items() for k1,v1 in v0.items()],
          "text"  : [v1 for k0,v0 in data_row.items() for k1,v1 in v0.items()]}
    docs_df = pd.DataFrame(data)
    
    index_ref = indexer.index(docs_df["text"], docs_df["docno"])

else:
    # dans le cas où l'index existe déjà
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
index = pt.IndexFactory.of(index_ref)

In [5]:
print(index.getCollectionStatistics().toString())

Number of documents: 50612
Number of terms: 103283
Number of postings: 3056988
Number of fields: 0
Number of tokens: 3906884
Field names: []
Positions:   false



In [4]:
df_train = pd.read_csv("WikiPassageQA/train.tsv",sep='\t')
df_test = pd.read_csv("WikiPassageQA/test.tsv",sep='\t')
df_dev = pd.read_csv("WikiPassageQA/dev.tsv",sep='\t')
df_query = pd.concat([df_train, df_test, df_dev])
df_query

,QID,Question,DocumentID,DocumentName,RelevantPassages
0,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4
1,195,How did the assault on the Bastille the first ...,359,French_Revolution.html,"1,2"
2,557,What is the prehistory of Albania?,285,Albania.html,4
3,1508,What significance did Bulgaria have in the end...,579,Central_Powers.html,14
4,956,What is the rationale of support of the Common...,204,Common_Era.html,9
...,...,...,...,...,...
412,343,Why were nearly all Blacks unable to vote in A...,822,Alabama.html,"17,18"
413,1875,How is the crime in Cleveland?,158,Cleveland.html,56
414,4079,What was the purpose of the one-party state?,78,Soviet_Union.html,13
415,1401,What distinguishes GDP from GNI?,424,Gross_domestic_product.html,13


In [5]:
import json
with open('WikiPassageQA/document_passages.json', 'r') as f:
        data_row = json.load(f)
data={"DocumentID" : [int(k0) for k0,v0 in data_row.items() for k1,v1 in v0.items()],
    "docno" : [k0+','+k1 for k0,v0 in data_row.items() for k1,v1 in v0.items()],
      "text"  : [v1 for k0,v0 in data_row.items() for k1,v1 in v0.items()]}
docs_df = pd.DataFrame(data)

In [6]:
docs_df.head()

,DocumentID,docno,text
0,344,"344,1",Townspeople who lived in chartered towns were ...
1,344,"344,0",A municipal corporation is the legal term for ...
2,344,"344,3","This standard varies from state to state, acco..."
3,344,"344,2",All municipal corporations must also abide by ...
4,344,"344,5","Under the panchayati raj system, it interacts ..."


## merge avec passage

In [7]:
df_complete = df_query.merge(docs_df, on =["DocumentID"])
df_complete

,QID,Question,DocumentID,DocumentName,RelevantPassages,docno,text
0,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,42",They claimed major credit for the election of ...
1,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,48",Evangelicalism was a major force in the Anglic...
2,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,43",The rich and the poor remained traditional Cat...
3,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,49","The Evangelical Alliance, formed in 1846, was ..."
4,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,24","By 1737, Whitefield had become a national cele..."
...,...,...,...,...,...,...,...
244131,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,72",The budgetary rule is to spend no more than 4%...
244132,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,71",Norway is the fifth-largest oil exporter and t...
244133,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,70","In 2011, 28% of state revenues were generated ..."
244134,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,79",The two most prominent are the E6 going north-...


## creer qrel

In [141]:
# cree list relevant
relevant = []
for row in df_complete.iterrows():
  passage = row[1]['RelevantPassages'].split(",")
  doc_passage = row[1]['docno'].split(",")[1] # doc,pass -> [1]
  if doc_passage in passage:
    relevant.append(1)
  else:
    relevant.append(0)

df_complete["label"] = relevant
qrel = df_complete[["QID", "docno","label"]]
qrel.columns = ['qid', 'docno', 'label']
qrel = qrel.astype({"qid": str, "docno": object, "label":object}) #qid en string pour pt.Experiment
qrel.head()

,qid,docno,label
0,3086,"672,42",0
1,3086,"672,48",0
2,3086,"672,43",0
3,3086,"672,49",0
4,3086,"672,24",0


In [135]:
topics = df_query[["QID", "Question"]]
topics.columns = ['qid', 'query']
topics

,qid,query
0,3086,What is the role of conversionism in Evangelic...
1,195,How did the assault on the Bastille the first ...
2,557,What is the prehistory of Albania?
3,1508,What significance did Bulgaria have in the end...
4,956,What is the rationale of support of the Common...
...,...,...
412,343,Why were nearly all Blacks unable to vote in A...
413,1875,How is the crime in Cleveland?
414,4079,What was the purpose of the one-party state?
415,1401,What distinguishes GDP from GNI?


In [124]:
import string
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer


def prepretraiter(query):
    #wn_lemmatizer = nltk.stem.WordNetLemmatizer()
    stemmer = SnowballStemmer("english")
    stopwords_en = stopwords.words('english')
    punc = string.punctuation
    q_traite = []
    for q in query:
        q = q[0].lower()
        q = q.translate(str.maketrans(punc, ' '*len(punc)))
        q = word_tokenize(q)
        q = ' '.join([w for w in q if w not in stopwords_en])
        #q = ' '.join(map(lambda x: wn_lemmatizer.lemmatize(x), word_tokenize(q)))
        q = ' '.join(map(lambda x: stemmer.stem(x), word_tokenize(q)))
        q_traite.append(q)
    return q_traite

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [125]:
liste_query = topics[['query']].values.tolist()
liste_query_traite = prepretraiter(liste_query)

In [137]:
topics['query'] = liste_query_traite
topics = topics.astype({"qid": str, "query": object})
topics

<ipython-input-137-4b75552aadb0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topics['query'] = liste_query_traite


,qid,query
0,3086,role convers evangel
1,195,assault bastil first year revolut ultim culmin...
2,557,prehistori albania
3,1508,signif bulgaria end world war
4,956,ration support common era
...,...,...
412,343,near black unabl vote alabama civil right act ...
413,1875,crime cleveland
414,4079,purpo one parti state
415,1401,distinguish gdp gni


## Sur data test

In [140]:
topic_test = df_test[["QID", "Question"]]
topic_test.columns = ['qid', 'query']

liste_query_test = topic_test[['query']].values.tolist()
liste_query_test_traite = prepretraiter(liste_query_test)
topic_test['query'] = liste_query_test_traite

topic_test = topic_test.astype({"qid": str, "query": object}) #string key
topic_test.head()

<ipython-input-140-a649801d2d0b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topic_test['query'] = liste_query_test_traite


,qid,query
0,449,iraq role polit unstabil
1,1140,warmest year global warm
2,782,olymp game broadcast
3,1421,elect turnout 2008
4,3738,secular misinterpret america


### VSM

In [142]:
wc = pt.BatchRetrieve(index, wmodel="Tf")
wc_idf = pt.BatchRetrieve(index, wmodel="TF_IDF", norm = None)
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
ql = pt.BatchRetrieve(index, wmodel="DirichletLM")


pt.Experiment(
    [wc, wc_idf, tf_idf, bm25, ql],
    topic_test,
    qrel,
    eval_metrics=["map", "recip_rank", "P_5","P_10","ndcg", "recall_5", "recall_10", "recall_20" ],
    names = ["WC", "WC.IDF", "VSM", "BM25", "QL"]
)

BR(TF_IDF):   0%|          | 0/416 [00:00<?, ?q/s]

,name,map,recip_rank,P_5,P_10,ndcg,recall_5,recall_10,recall_20
0,WC,0.120860,0.145953,0.044231,0.032692,0.271955,0.160857,0.229768,0.383814
1,WC.IDF,0.386969,0.461777,0.136538,0.082933,0.519434,0.463742,0.541947,0.647716
2,VSM,0.386969,0.461777,0.136538,0.082933,0.519434,0.463742,0.541947,0.647716
3,BM25,0.388273,0.464128,0.135577,0.082692,0.520172,0.459415,0.540745,0.642508
4,QL,0.333991,0.404189,0.121154,0.073798,0.471931,0.417147,0.496875,0.595433
